<a href="https://colab.research.google.com/github/YuehMintTai/000/blob/main/MongoDBTest20220201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install flask_ngrok
!pip install faker
!pip install pdfkit

In [2]:
###https://dashboard.ngrok.com/get-started/setup!mkdir -p /drive/ngrok-ssh
##https://colab.research.google.com/github/shawwn/colab-tricks/blob/master/ngrok-tricks.ipynb#scrollTo=1B_aC562JbJU
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok
!./ngrok authtoken 24VntdRE0Mqj6buTgjDDBLZLSBN_4tX4P8h72ZYpEudaobfKd

[Errno 2] No such file or directory: '/drive/ngrok-ssh'
/content
--2022-02-02 11:10:47--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  47.1MB/s    in 0.3s    

2022-02-02 11:10:48 (47.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
cp: cannot stat '/drive/ngrok-ssh/ngrok': No such file or directory
chmod: cannot access '/ngrok': No such file or directory
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [43]:
from flask.templating import render_template_string
from flask_ngrok import run_with_ngrok
from flask import Flask,request,make_response,Response,render_template
import pymongo,json,faker,base64,io,pdfkit,datetime
from PIL import Image
from bson.objectid import ObjectId

app=Flask(__name__)
run_with_ngrok(app)

fake=faker.Faker('zh-TW')
myText=''
try:
  mongo=pymongo.MongoClient('mongodb://mymongodbtry:7dmrAHuHW7lQdVRhcEUFtTowlGEaiLJua4jFh04EM3CbcVJMrW02uZrkTygJX6hpdsRw1fo6PP67x2vTbMj8mQ==@mymongodbtry.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@mymongodbtry@')
  mycol=mongo.myTest.test6
  myText='Succeed in connecting mongodb.'
except Exception as ex:
  print(ex)
  myText='Fail to connect mongodb.'
######LLLLLLLLLLLLLLLLLLLLLLLLLLLL
@app.route('/')
def home():
  try:
    mylist=list(mycol.find().sort([('_id',-1)]))
    for j,i in enumerate(mylist):
      i['_id']=str(i['_id'])
      i['序號']=str(j+1)
    ##return myText
    #return json.dumps(mylist,ensure_ascii=False).encode('utf-8') 
    return render_template('index.html',title='User List',jsonfile=mylist)
  except Exception as ex:
    return ex
  ##return '哈囉你好嗎?'
#####CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
@app.route('/createUser/')
def createUser():
  try:
    newUser={'姓名':f'{fake.name()}',
             'email':f'{fake.email()}',
             '地址':f'{fake.address()}',
             '留言':f'{fake.text()}'}
    #print(newUser)
    #return json.dumps(newUser,ensure_ascii=False,indent=4).encode('utf-8')
    myResponse=mycol.insert_one(newUser)
    return f'Succeed in creating user id:{myResponse.inserted_id}.<a href="/"> /BACK LIST/ </a>'
  except Exception as ex:
    return ex
####RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
@app.route('/readUser/<id>')
def readUser(id):
  try:
    mylist=list(mycol.find({'_id':ObjectId(id)}))
    for i in mylist:
      i['_id']=str(i['_id'])
    return render_template('form.html',title='READ USER',jsonfile=mylist)
  except Exception as ex:
    return ex
###UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU
@app.route('/updateUser/', methods=['POST'])
def updateUser():
  try:
    myID=request.form['_id']
    myName=request.form['姓名']
    myEmail=request.form['email']
    myAddress=request.form['地址']
    myContent=request.form['留言']
    myFilter={'_id':ObjectId(myID)}
    myValue={'$set':{'姓名':myName,'email':myEmail,'地址':myAddress,'留言':myContent}}
    mycol.update_one(myFilter,myValue)
    return f'update id:{myID},<a href=/> BACK </a>'
  except Exception as ex:
    return ex

###DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
@app.route('/deleteUser/<id>')
def deleteUser(id):
  try: 
    myResponse=mycol.delete_one({'_id':ObjectId(id)})
    return f'Deleting user: {id}. <a href="/"> BACK LIST </a>' 
  except Exception as ex:
    return ex
#####FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
if __name__=='__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e701-34-74-103-58.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/Feb/2022 14:24:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 14:24:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Feb/2022 14:24:15] "GET /readUser/61fa881681aca02dd79abae0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 14:24:22] "POST /updateUser/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 14:24:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 14:24:33] "GET /deleteUser/61fa881681aca02dd79abae0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 14:24:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 14:24:39] "GET /readUser/61fa87b681aca02dd79abade HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 14:24:45] "POST /updateUser/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 14:24:47] "GET / HTTP/1.1" 200 -
